In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset
dataset = pd.read_csv('mushroom.csv')

# Independent variables (features) and dependent variable (target)
X = dataset.iloc[:, 1:].values  # All features except "class"
Y = dataset.iloc[:, 0].values  # The poisonous/edible feature

# Handle missing values with SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
X = imputer.fit_transform(X)

# Use LabelEncoder to convert categorical features to integer labels
label_encoders = []
for col in range(X.shape[1]):
    le = LabelEncoder()  # Create a new label encoder for each column
    X[:, col] = le.fit_transform(X[:, col])  # Transform each column using the label encoder
    label_encoders.append(le)

# Label encode the target variable
y_encoder = LabelEncoder()
Y = y_encoder.fit_transform(Y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0, shuffle=True)

# Feature scaling
scaler = StandardScaler(with_mean=False)  # Avoid centering sparse data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dropout(0.2),  # Dropout for regularization
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train the model on the training set
model = build_model()
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary output

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score (Weighted):", f1)

# Classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store cross-validation results
accuracy_scores = []
f1_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(X):
    # Split data into training and testing sets
    X_train_cv, X_test_cv = X[train_index], X[test_index]
    y_train_cv, y_test_cv = Y[train_index], Y[test_index]
    
    # Scale the features
    X_train_cv = scaler.fit_transform(X_train_cv)
    X_test_cv = scaler.transform(X_test_cv)
    
    # Train and evaluate the model for each fold
    model_cv = build_model()
    model_cv.fit(X_train_cv, y_train_cv, epochs=10, batch_size=32, validation_split=0.1, verbose=1)
    
    # Predictions for cross-validation
    y_pred_cv = (model_cv.predict(X_test_cv) > 0.5).astype(int)
    
    # Store accuracy and F1 scores for this fold
    accuracy_cv = accuracy_score(y_test_cv, y_pred_cv)
    f1_cv = f1_score(y_test_cv, y_pred_cv, average='weighted')
    
    # Add to lists
    accuracy_scores.append(accuracy_cv)
    f1_scores.append(f1_cv)

# Cross-validation results
print("Cross-Validation Accuracy Scores:", accuracy_scores)
print("Mean Cross-Validation Accuracy:", np.mean(accuracy_scores))
print("Mean Cross-Validation F1 Score:", np.mean(f1_scores))


Epoch 1/10


C:\Users\c g\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1203/1203 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6930 - loss: 0.5573 - val_accuracy: 0.8954 - val_loss: 0.2523
Epoch 2/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8874 - loss: 0.2651 - val_accuracy: 0.9497 - val_loss: 0.1312
Epoch 3/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9321 - loss: 0.1652 - val_accuracy: 0.9719 - val_loss: 0.0801
Epoch 4/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9494 - loss: 0.1238 - val_accuracy: 0.9698 - val_loss: 0.0752
Epoch 5/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9604 - loss: 0.1022 - val_accuracy: 0.9724 - val_loss: 0.0734
Epoch 6/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9644 - loss: 0.0892 - val_accuracy: 0.9853 - val_loss: 0.0448
Epoch 7/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9720 - loss: 0.0750 - val_accuracy: 0.9869 - val_loss: 0.0366
Epoch 8/10
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9714 - loss: 0.0713 - val_accurac

C:\Users\c g\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1375/1375 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6964 - loss: 0.5500 - val_accuracy: 0.5939 - val_loss: 1.2718
Epoch 2/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8900 - loss: 0.2472 - val_accuracy: 0.6582 - val_loss: 2.0249
Epoch 3/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9406 - loss: 0.1480 - val_accuracy: 0.6824 - val_loss: 2.4963
Epoch 4/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9557 - loss: 0.1128 - val_accuracy: 0.6993 - val_loss: 3.2216
Epoch 5/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9640 - loss: 0.0912 - val_accuracy: 0.6922 - val_loss: 3.6447
Epoch 6/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9698 - loss: 0.0780 - val_accuracy: 0.6774 - val_loss: 3.2646
Epoch 7/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9755 - loss: 0.0649 - val_accuracy: 0.6500 - val_loss: 4.2449
Epoch 8/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9753 - loss: 0.0665 - val_accurac

...

1375/1375 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6949 - loss: 0.5544 - val_accuracy: 0.6969 - val_loss: 0.8817
Epoch 2/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8962 - loss: 0.2402 - val_accuracy: 0.6791 - val_loss: 1.4913
Epoch 3/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9407 - loss: 0.1505 - val_accuracy: 0.6885 - val_loss: 1.8436
Epoch 4/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9585 - loss: 0.1108 - val_accuracy: 0.7077 - val_loss: 2.1601
Epoch 5/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9664 - loss: 0.0891 - val_accuracy: 0.6719 - val_loss: 2.7365
Epoch 6/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9734 - loss: 0.0733 - val_accuracy: 0.6230 - val_loss: 3.2099
Epoch 7/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9761 - loss: 0.0641 - val_accuracy: 0.6109 - val_loss: 3.6688
Epoch 8/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9788 - loss: 0.0611 - val_accurac

C:\Users\c g\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1375/1375 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6780 - loss: 0.5846 - val_accuracy: 0.5211 - val_loss: 2.2207
Epoch 2/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8900 - loss: 0.2547 - val_accuracy: 0.5106 - val_loss: 4.2975
Epoch 3/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.1569 - val_accuracy: 0.5577 - val_loss: 4.7738
Epoch 4/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9530 - loss: 0.1193 - val_accuracy: 0.5129 - val_loss: 6.3717
Epoch 5/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9619 - loss: 0.0995 - val_accuracy: 0.5745 - val_loss: 7.1580
Epoch 6/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9624 - loss: 0.0975 - val_accuracy: 0.5708 - val_loss: 7.9742
Epoch 7/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9726 - loss: 0.0717 - val_accuracy: 0.6027 - val_loss: 6.6279
Epoch 8/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9746 - loss: 0.0673 - val_accurac

C:\Users\c g\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6999 - loss: 0.5449 - val_accuracy: 0.6326 - val_loss: 1.1786
Epoch 2/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8970 - loss: 0.2432 - val_accuracy: 0.6459 - val_loss: 1.4694
Epoch 3/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9379 - loss: 0.1530 - val_accuracy: 0.5886 - val_loss: 2.5273
Epoch 4/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9552 - loss: 0.1151 - val_accuracy: 0.6279 - val_loss: 2.7430
Epoch 5/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9614 - loss: 0.0960 - val_accuracy: 0.6609 - val_loss: 3.1160
Epoch 6/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9632 - loss: 0.0960 - val_accuracy: 0.6447 - val_loss: 3.2125
Epoch 7/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9740 - loss: 0.0689 - val_accuracy: 0.6330 - val_loss: 3.0538
Epoch 8/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9744 - loss: 0.0657 - val_accurac